In [ ]:
import os
# import shutil
import warnings
# import sys
import glob
import re
# import math
from datetime import datetime
# from datetime import timedelta
# from dateutil.relativedelta import relativedelta
from collections import defaultdict
from collections import OrderedDict
# from scipy import stats
# import scipy.io
import csv
import xlsxwriter
import time
from tqdm.notebook import tqdm_notebook
import numpy as np
import pandas as pd
# import seaborn as sns
import openpyxl as xl
from copy import copy
from copy import deepcopy
# from openpyxl import load_workbook
%matplotlib widget
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec
# from matplotlib.font_manager import FontProperties
# import statsmodels.api as sm
# from bioinfokit.analys import stat
# from statsmodels.formula.api import ols
# import pingouin as pg
# from statsmodels.graphics.factorplots import interaction_plot
# from brokenaxes import brokenaxes
import matplotlib.patheffects as pe
# from matplotlib import container
# import concurrent.futures
# from itertools import starmap
# import multiprocessing as mp
# from brokenaxes import brokenaxes

In [ ]:
### hide these types of warning
warnings.filterwarnings(action='default', message='.*converting a masked element to nan.*')

### Display all columns in pandas dataframe by default
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### set path
path =  os.getcwd()
print('path is:', path, '\n')

foldername = os.path.basename(path)
excelName = os.path.join(path, foldername + '.xlsx')
print('foldername is:', foldername,'\n')
print('excelName is:', excelName)
os.chdir(path)

### Some plotting presets
plt.rc("errorbar", capsize=3)
plt.rc("figure", dpi=100)
plt.rc("savefig", dpi=220, facecolor="white", bbox="tight") # 

In [ ]:
### Obtain treatments, concentrations and well coloring from ac spike_list file
ac_spike_file = [i for i in os.listdir() if 'spike' in i and 'ac' in i][0]
with open(ac_spike_file, 'r', encoding='UTF-8') as csv_file:
    row = list(csv.reader(csv_file, delimiter=','))
    for lin in tqdm_notebook(row[-7:-1], desc = "Obtaining treatments, concentrations and well coloring from spike_list files on " + str(time.ctime(time.time()))):
        if lin[0] == 'Well':
            wells = [str(i) for i in lin] # string list of treatments
        elif lin[0] == 'Treatment':
            treats = [str(i) for i in lin] # string list of treatments
        elif lin[0] == 'Concentration':
            concs = [str(i) for i in lin] # string list of concentrations
        elif lin[0] == 'Well Coloring':
            colorings = [str(i) for i in lin] # list of well colors
print(wells,treats,concs,colorings, sep='\n\n')

In [ ]:
### Delete 'sorted ' from file names
for file in glob.glob('*supp.csv'):
    os.rename(file,file.replace('sorted ', '').replace('__','_'))
###  Add this obtained well info to supp files
supp_files = [i for i in os.listdir() if '_supp' in i]
reader = []
cont = 0
for fname in glob.glob('*supp.csv'):
    with open(fname, "r", encoding='UTF-8') as infile:
        reader = list(csv.reader(infile, delimiter=','))
        if reader[29][0] != 'Treatment':
                reader.insert(29, colorings)
                reader.insert(29, concs)
                reader.insert(29, treats)
                with open(fname, "w", newline='', encoding='UTF-8') as outfile:
                    writer = csv.writer(outfile)
                    for line in reader:
                        writer.writerow(line)
        else:
            print('{} already contains well info'.format(fname))
            cont += 1

print('\n{} supp files out of {} found contain well info'.format(cont, len(glob.glob('*supp.csv'))))

In [ ]:
### Couple each file with its date of creation, retrieved from each of their corresponding _spike_file(s)
spikedatesdict = OrderedDict()
suppdatesdict = OrderedDict()
wfdatesdict = OrderedDict()
with tqdm_notebook(total=len([f for f in os.listdir() if f.split('.')[-1] == 'csv' and '_spike_list' in f]),
                               desc="Coupling each file with its date of creation, retrieved from _spike_file(s) on " + str(time.ctime(time.time()))) as pbar:
    for fl in os.listdir():
        if fl.split('.')[-1] == 'csv' and '_spike_list' in fl:
            with open(fl, encoding='UTF-8') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                row = list(csv_reader)
                for lin in row[4:6]:
                    if lin[0] == '   Original File Time':
                        pbar.update(1)
                        spikedatesdict[fl] = lin[1]
                        suppdatesdict[fl.replace('_spike_list','_neuralMetrics_supp')] = lin[1]
                        wfdatesdict[fl.replace('_spike_list','.wfexp').replace('csv','txt')] = lin[1]
                    
print('spikedatesdict is:\n') 
for k, v in spikedatesdict.items():
    print(k, v)
print('\nsuppdatesdict is:\n') 
for k, v in suppdatesdict.items():
    print(k, v)
print('\nwfdatesdict is:\n') 
for k, v in wfdatesdict.items():
    print(k, v)

In [ ]:
# Sort file names by their date of creation
spikedatesdict = dict(sorted(spikedatesdict.items(), key=lambda x: datetime.strptime(x[1], '%m/%d/%Y %H:%M:%S')))
suppdatesdict = dict(sorted(suppdatesdict.items(), key=lambda x: datetime.strptime(x[1], '%m/%d/%Y %H:%M:%S')))
wfdatesdict = dict(sorted(wfdatesdict.items(), key=lambda x: datetime.strptime(x[1], '%m/%d/%Y %H:%M:%S'))) # got to get the order from spike_list files because only there figures the timestamp
# Clean file names for the excel workbook and graphs
newdatesdict = OrderedDict()
for key, value in suppdatesdict.items():
    if 'on' in key and '(000)(000)' in key:
        new_key = key.split('.')[0].replace('(000)(000)', ' 0').replace('_neuralMetrics_supp','')
        newdatesdict[new_key] = value
    else:
        new_key = key.split('.')[0].replace('(000)', '').replace('(00', ' ').replace('(0', ' ').replace(')', '').replace('_neuralMetrics_supp','')
        newdatesdict[new_key] = value
for i, (k, v) in enumerate(newdatesdict.items()):
    print('newdatesdict {} is: {} {}'.format(i, k, v))

In [ ]:
### append desired parameters' data to dataList from the advanced metrics files
rowsChulas = np.r_[28,29,30,31,33:39 + 1,41:68 + 1,70:98 + 1,100:105 + 1]
dataList = []
duration_in_sec = False
number_of_wells = None

for fl in suppdatesdict.keys():
#     if fl.split('.')[-1] == 'csv' and '_' not in fl:            
    with open(fl, encoding='UTF-8') as csv_file:
        singleCSV = []
        csv_reader = csv.reader(csv_file, delimiter=',')
        for i, row_content in enumerate(csv_reader):
            if i == 67:
                number_of_wells = len(row_content)
            if row_content != [] and row_content[0] == 'Analysis End':
                duration_in_sec = float(row_content[1])
            if i in rowsChulas:
                row_content[0] = row_content[0].strip()
                if row_content[0] == 'Well Averages':
                    row_content.pop(-1) ## this line ends with a "," making it one position longer as a CSV file
                if (row_content[0] == 'Number of Spikes' or row_content[0] == 'Number of Bursts' or
                row_content[0] == 'Number of Network Bursts' or row_content[0] == 'Inter-Burst Interval - Avg (s)' or
                row_content[0] == 'Inter-Burst Interval - Std (s)'):
                    singleCSV.append([round(float(x) / (duration_in_sec/660)) if x != '' else '' for x in row_content[1:]]) # pseudo-normalize by time span of the recording
                    singleCSV[-1].insert(0, row_content[0])
                elif row_content[0] == 'Mean Firing Rate (Hz)':
                    if len(number_of_wells) > 7:
                        singleCSV.append([str(float(a)*16) for a in row_content[1:]])
                    elif len(number_of_wells) == 7:
                        singleCSV.append([str(float(a)*16) for a in row_content[1:]])
                    singleCSV[5].insert(0, 'Mean Firing Rate (Hz) - well')
                    singleCSV.append(row_content)
                    singleCSV[6][0] = 'Mean Firing Rate (Hz) - elec'
                elif row_content[0] == 'Weighted Mean Firing Rate (Hz)':
                    singleCSV.append([str(float(a)*16) if a != ' ' else ' ' for a in row_content[1:]])
                    singleCSV[11].insert(0, 'Weighted Mean Firing Rate (Hz) - unit')
                    singleCSV.append(row_content)
                    singleCSV[12][0] = 'Weighted Mean Firing Rate (Hz) - elec'
                elif row_content[0] == 'Number of Bursting Units':
                    singleCSV.append(row_content)
                    b = row_content[1:]                    
                elif row_content[0] == 'Burst Frequency - Avg (Hz)':
                    singleCSV.append([str(float(a)/float(b)) if a != ' ' and b != ' ' else 'NaN' for a, b in zip(row_content[1:],b)])
                    singleCSV[33].insert(0, 'Burst Frequency per unit - Avg (Hz)')
                elif row_content[0] == 'Burst Frequency - Std (Hz)':
                    singleCSV.append([str(float(a)*16) if a != ' ' else ' ' for a in row_content[1:]])
                    singleCSV[34].insert(0, 'Burst Frequency per unit - Std (Hz)')
                else:
                    singleCSV.append(row_content)
        dataList.append(singleCSV)
for i, row_content in enumerate(dataList):
    dataList[i][1][1:] = [j + ' ' + i if j != '' else i for i, j in zip(dataList[i][1][1:], dataList[i][2][1:])]
    
'''In the future extract the concentration of the second compound used in the well from the name too
(use "after the + symbol" regular expression to find it) and add it to the "concentration" row in dataList'''

In [ ]:
### manually define a dictionary of shortened parameter names for the excel sheet names

excel_friendly = {
'Well Averages':'Well Averages',
'Treatment':'Treatment',
'Concentration':'Concentration',
'Well Coloring':'Well Coloring',
'Number of Spikes':'Number_of_Spikes',
'Mean Firing Rate (Hz) - well':'Mean_Firing_Rate_Hz_well',
'Mean Firing Rate (Hz) - elec':'Mean_Firing_Rate_Hz_elec',
'ISI Coefficient of Variation - Avg':'ISI_Coeff_of_Variation_Avg',
'Network ISI Coefficient of Variation':'Network_ISI_Coeff_Variation',
'Fano Factor': 'Fano_Factor',
'Number of Active Units':'Number_of_Active_Units',
'Weighted Mean Firing Rate (Hz) - unit':'Weighted_Mean_Spk_Rate_Hz_unit',
'Weighted Mean Firing Rate (Hz) - elec':'Weighted_Mean_Spk_Rate_Hz_elec',
'Number of Bursts':'Number_of_Bursts',
'Number of Bursting Units':'Number_of_Bursting_Units',
'Burst Duration - Avg (sec)':'Burst_Duration_Avg_sec',
'Burst Duration - Std (sec)':'Burst_Duration_Std_sec',
'Burst Duration (Median) - Avg (sec)':'Burst_Duration_Med_Avg_s',
'Burst Duration (Median) - Std (sec)':'Burst_Duration_Med_Std_s',
'Number of Spikes per Burst - Avg':'Spikes_per_Burst_Avg',
'Number of Spikes per Burst - Std':'Spikes_per_Burst_Std',
'Number of Spikes per Burst (Median) - Avg':'Spikes_per_Burst_Med_Avg',
'Number of Spikes per Burst (Median) - Std':'Spikes_per_Burst_Med_Std',
'Mean ISI within Burst - Avg (sec)':'Mean_ISI_within_Burst_Avg_s',
'Mean ISI within Burst - Std (sec)':'Mean_ISI_within_Burst_Std_s',
'Median ISI within Burst - Avg (sec)':'Median_ISI_within_Burst_Avg_s',
'Median ISI within Burst - Std (sec)':'Median_ISI_within_Burst_Std_s',
'Median/Mean ISI within Burst - Avg':'x̃÷x̅_ISI_within_Burst_Avg',
'Median/Mean ISI within Burst - Std':'x̃÷x̅_ISI_within_Burst_Std',
'Inter-Burst Interval - Avg (sec)':'Inter_Burst_Interval_Avg_s',
'Inter-Burst Interval - Std (sec)':'Inter_Burst_Interval_Std_s',
'Inter-Burst Interval (Median) - Avg (sec)':'Inter_Burst_Interv_x̃_Avg_s',
'Inter-Burst Interval (Median) - Std (sec)':'Inter_Burst_Interv_x̃_Std_s',
'Burst Frequency per unit - Avg (Hz)':'Burst_Frequency_unit_Avg_Hz',
'Burst Frequency per unit - Std (Hz)':'Burst_Frequency_unit_Std_Hz',
'Normalized Duration IQR - Avg':'Normalized_Duration_IQR_Avg',
'Normalized Duration IQR - Std':'Normalized_Duration_IQR_Std',
'IBI Coefficient of Variation - Avg':'IBI_Coeff_of_Variation_Avg',
'IBI Coefficient of Variation - Std':'IBI_Coeff_of_Variation_Std',
'Burst Percentage - Avg':'Burst_Percentage_Avg',
'Burst Percentage - Std':'Burst_Percentage_Std',
'Network Bursts Ignored Flag':'Network_Bursts_Ignored_Flag',
'Number of Network Bursts':'Number_of_Network_Bursts',
'Network Burst Frequency':'Network_Burst_Frequency_Hz',
'Network Burst Duration - Avg (sec)':'Network_Burst_Duration_Avg_s',
'Network Burst Duration - Std (sec)':'Network_Burst_Duration_Std_s',
'Network Burst Duration - Median (sec)':'Network_Burst_Duration_Med_s',
'Network Burst Duration - MAD (sec)':'Network_Burst_Duration_MAD_s',
'Number of Spikes per Network Burst - Avg':'Spikes_per_Netw_Burst_Avg',
'Number of Spikes per Network Burst - Std':'Spikes_per_Netw_Burst_Std',
'Number of Spikes per Network Burst - Median':'Spikes_per_Netw_Burst_Med',
'Number of Spikes per Network Burst - MAD':'Spikes_per_Netw_Burst_MAD',
'Mean ISI within Network Burst - Avg (sec)':'x̅_ISI_within_Netw_Burst_Avg_s',
'Mean ISI within Network Burst - Std (sec)':'x̅_ISI_within_Netw_Burst_Std_s',
'Median ISI within Network Burst - Avg (sec)':'x̃_ISI_within_Netw_Burst_Avg_s',
'Median ISI within Network Burst - Std (sec)':'x̃_ISI_within_Netw_Burst_Std_s',
'Median/Mean ISI within Network Burst - Avg':'x̃÷x̅_ISI_within_Netw_Brst_Avg',
'Median/Mean ISI within Network Burst - Std':'x̃÷x̅_ISI_within_Netw_Brst_Std',
'ISI CoV within Network Burst - Avg':'ISI_CoV_within_Netw_Burst_Avg',
'Number of Units Participating in Burst - Avg':'Units_Particip_in_Brst_Avg',
'Number of Units Participating in Burst - Std':'Units_Particip_in_Brst_Std',
'Number of Units Participating in Burst - Median':'Units_Particip_in_Brst_Med',
'Number of Units Participating in Burst - MAD':'Units_Particip_in_Brst_MAD',
'Number of Spikes per Network Burst per Unit - Avg':'Spikes_per_NwBrst_per_Unit_Avg',
'Number of Spikes per Network Burst per Unit - Std':'Spikes_per_NwBrst_per_Unit_Std',
'Number of Spikes per Network Burst per Unit - Median':'Spikes_per_NwBrst_per_Unit_Med',
'Number of Spikes per Network Burst per Unit - MAD':'Spikes_per_NwBrst_per_Unit_MAD',
'Network Burst Percentage':'Network_Burst_Percentage',
'Network IBI Coefficient of Variation':'Network_IBI_Coeff_Variation',
'Network Normalized Duration IQR':'Network_Norm_Duration_IQR',
'Area Under Normalized Cross-Correlation':'Area_Under_Norm_Cross_Corr',
'Area Under Cross-Correlation':'Area_Under_Cross_Correlation',
'Full Width at Half Height of Normalized Cross-Correlation':'Width_Half_Height_N_Cross_Corr',
'Full Width at Half Height of Cross-Correlation':'Width_Half_Height_Cross_Corr',
'Synchrony Index':'Synchrony_Index',
'Kreuz SPIKE Distance':'Kreuz_SPIKE_Distance'
}

### Make sure all names are <=31 chars long
for i in excel_friendly.values():
    if len(i) > 31:
        print(i+ ' is too long of an excel worksheet name')
print('Number of advanced metrics parameters =', len(excel_friendly))

In [ ]:
### assign each row in dataList to its day in newdatesdict, load into data dictionary
data = defaultdict(dict)

for day, ds in zip(newdatesdict.keys(), dataList):
    for row in ds:
        key = excel_friendly[row[0]]
        data[day][key] = row[1:]

for day in newdatesdict.keys():
    keys = list(data[day].keys())
    vals = list(data[day].values())
    idx = np.argsort(data[day]['Treatment'])#merging treatment and concentration values within 'Treatment' key allows to easily sort columns by both at the same time,
                                            #useful when different concentrations of the same treatment are present in the same plate
print('\nIndex of sorted well treatments is: ', idx)

In [ ]:
### find the first time point that received a treatment and the last that did not, and save their names as "baseline" and "acute", respectively
# Use when there is only one baseline and one acute time point
baseline = False
day_of_treatment = False
for i, day in enumerate(data):
    for hexcolor in data[day]['Well Coloring']:
        if hexcolor != '#00FF00' and 'bl' in day:
            baseline = day
            break
    else:
        continue
    break
acute = [k for k in list(newdatesdict.keys()) if 'ac' in k and 'ac(' not in k][0]
day_of_treatment = acute.split(' ')[0].replace('div','')
print(baseline, acute, '\Day of the treatment is: ', day_of_treatment)

In [ ]:
### Calculate relative time from each time point to the acute treatment (time 0)
def_labels = {}
realistic_x_axis = {}
act = datetime.strptime(newdatesdict[acute], '%m/%d/%Y %H:%M:%S')
for key, value in newdatesdict.items():
    now = datetime.strptime(newdatesdict[key], '%m/%d/%Y %H:%M:%S')
    diff = now - act
    hours = diff.days*24 + diff.seconds/3600
    minutes = abs(hours*60) % 60
    seconds = abs(hours*3600) % 60
    def_labels[key] = ["{}: {}".format(key, "%d%s%02d%s%02d%s" % (hours,'h ',minutes,'min ',seconds,'sec'))]
    realistic_x_axis[key] = diff.days*24 + diff.seconds/3600

print('def_labels:\n')
for k, v in def_labels.items():
    print(k+':\t', ",".join(v))

print('\nrealistic_x_axis:\n')
for k, v in realistic_x_axis.items():
    print(k+':\t', v, 'hours')

In [ ]:
### First, calculate the average of all baseline timepoints when
### needed, then append baseline values for normalization to norms
### dictionary if there is at least one 'bl' item in "data".
baselines = [key for key in data.keys() if 'bl' in key.lower()]
for element in baselines: # This would be the place to do the averaging of all the baseline time points
    norms = dict()
    for parameter, quantity in data[element].items(): 
        if parameter not in ['Well', 'Well Coloring', 'Treatment', 'Concentration']:
            norms[parameter] = [] # As it is written now, "norms" will contain the data from
                                  # the last baseline timepoint, usually bl(011) in a batch of 12 bl time points
            for val in quantity:
                try:
                    norms[parameter].append(float(val))
                except ValueError:
                    norms[parameter].append(None)

### List of all acute treatment timepoints
acutes = [key for key in data.keys() if 'ac' in key.lower()]
day_of_treatment = acutes[0].split(' ')[0].replace('div','')
print(baselines,acutes,day_of_treatment)

In [ ]:
### create excel workbook
excelName = os.path.join(output_dir, 'Tables_sorted_well_single.xlsx')
with xlsxwriter.Workbook(excelName, options= {'nan_inf_to_errors': True}) as workbook:
    with tqdm_notebook(total=len(data.values()), desc="Writing advanced metrics workbook rows on " + str(time.ctime(time.time()))) as pbar:
        styles = dict()
        params = data[list(newdatesdict.keys())[0]]
        sheets_names = []
        for key in params.keys():
            if key not in ['Well Averages', 'Well Coloring', 'Treatment', 'Concentration']:
                sheets_names.append(key)
        for name in sheets_names:
            workbook.add_worksheet(name)
            workbook.add_worksheet('n{}'.format(name))
        sheets = [sh for sh in workbook.worksheets() if sh.name in sheets_names]
        norm_sheets = [sh for sh in workbook.worksheets() if sh.name not in sheets_names]
        for i, day_data in enumerate(data.values()):
            pbar.update(1)
            for color, treat in zip(day_data['Well Coloring'], day_data['Treatment']):
                if not treat in styles:
                    styles[treat] = workbook.add_format({'bg_color': color})
                #styles[''] = workbook.add_format({'bg_color': '#00FF00'})#define absent treatment condition and its color
            for sh in sheets:
                sh.write(i + 2, 0, list(newdatesdict.keys())[i])
                sh.write(1, 0, 'Time')
                sh.write(0, 0, 'Well')
                for j in range(len(day_data['Well Averages'])):
                    sh.write(0, j + 1, day_data['Well Averages'][idx[j]])
                    treatment = day_data['Treatment'][idx[j]]
                    sh.write(1, j + 1, treatment)
                    raw = day_data[sh.name][idx[j]]
                    try:
                        val = float(raw)
                        if val == 0:
                            val = None
                    except ValueError as e:
    #                   print(str(e).replace(':', ' in') + " {}, {}, {}".format(list(newdatesdict.keys())[i],
                                                            #sh.name,
                                                            #day_data['Well'][idx[j]]))
                        val = None
                    sh.write(i + 2, j + 1, val, styles[day_data['Treatment'][idx[j]]])
            for sh in norm_sheets:
                sh.write(i + 2, 0, list(newdatesdict.keys())[i])
                sh.write(1, 0, 'Time')
                sh.write(0, 0, 'Well')
                for j in range(len(day_data['Well Averages'])):
                    sh.write(0, j + 1, day_data['Well Averages'][idx[j]])
                    treatment = day_data['Treatment'][idx[j]]
                    sh.write(1, j + 1, treatment)
                    raw = day_data[sh.name[1:]][idx[j]]
                    try:
                        val = float(raw) / norms[sh.name[1:]][idx[j]]
                        if val == 0:
                            val = None
                    except ValueError:
                        val = None
                    except TypeError:
                        val = None
                    except ZeroDivisionError: # if a well has active electrodes
#                          in the baseline, a ZeroDivisioError will prompt when
#                          trying to normalize (will try to divide float by 0)
#                          print("{} {} {} {}".format('Cannot divide',str(raw),'by',str(norms[sh.name[1:]][idx[j]])))
                        val = None
                    sh.write(i + 2, j + 1, val, styles[day_data['Treatment'][idx[j]]])
print('Created', excelName)
# """A FileNotFound error prompting here means the name is too long for an excel workbook and it could not be initialized"""

In [ ]:
'''Opening amplitude files...'''

In [ ]:
### create data structures
nombresListAmps = []
ampdata = dict()
well_info = dict()

### add ampdata to structures
for fname in tqdm_notebook(wfdatesdict.keys(),
                           desc = "Coupling amplitude values with their day, electrode, treatment and treatment concentration on " + str(time.ctime(time.time()))):
    # append short file names to "nombresListAmps"
    day = fname.split('.')[0].replace('on(000)(000)', 'on 0').replace('(000)', '').replace('.wfexp', '').replace('(00', ' ').replace('(0', ' ').replace(')', '')
    print('Processing:',day)
    nombresListAmps.append(day)
    with open(fname, encoding='UTF-8') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',') # execution time in this step is limited by disk data transfer speed,
            # specially by the size of the bl and ac spike_list files
        ampdata[day] = dict()
        ampdata[day]['Electrode'] = []
        ampdata[day]['Amplitude'] = []
        for i, row in enumerate(csv_reader):
            if i > 0:
                ampdata[day]['Electrode'].append(row[2])
                ampdata[day]['Amplitude'].append(float(row[11]))
        well, coloring, treatment, concentration = None, None, None, None # well info is at the bottom of the table, below the last amplitude values
        well = wells[1:]
        coloring = colorings[1:]
        treatment = treats[1:]
        concentration = concs[1:]
        well_info[day] = dict(zip(well, zip(coloring, treatment, concentration)))
print('\nCreated "ampdata" and "Well_info" dictionaries')

In [ ]:
### Converting amplitude values to μV and assigning them to their well of origin
well_data = dict()
for day, amp_day_data in tqdm_notebook(ampdata.items(),
                                       desc = "Converting each day\'s amplitude values to μV and assigning them to their well of origin on " + str(time.ctime(time.time()))):
    well_data[day] = defaultdict(list)
    for el, amp in zip(amp_day_data['Electrode'], amp_day_data['Amplitude']):
        well_data[day][el[:2]].append(amp/1000)

In [ ]:
### Sorting columns by treatment and concentration
#if 'ac' in well_data.keys(): # if treatment present in days list
treatments = np.array([(w, conc+' '+t) if conc != '' else (w, t) for w, (color, t, conc) in well_info[nombresListAmps[-1]].items()], dtype=str).T
idx = np.argsort(treatments)[1]
# else:
#     print('This is a development excel table')
#     treatments = np.array([(w, t) for w, (c, t) in well_info['div1'].items()], dtype=str).T
#     idx = np.argsort(treatments)[0]

print('\nEach well\'s treatments are: ',  treatments[1])
print('\nIndex of sorted well treatments is: ', idx)
print('\nWell treatments sorted by final table arrangement: ', treatments[1][idx])

col_numbers = dict(zip(treatments[0][idx], range(len(idx))))

In [ ]:
# Create excel workbook
excelNameAmps = os.path.join(path, 'amps.xlsx')
start = time.time()
with xlsxwriter.Workbook(excelNameAmps, options= {'nan_inf_to_errors': True}) as workbook:
    with tqdm_notebook(ncols=900, desc="Writing amplitude cells to workbook on " + str(time.ctime(time.time()))) as pbar:
    # Write raw worksheets
        sh = workbook.add_worksheet('Mean_Amplitude_μV')
        sh.write(1, 0, 'Time')
        sh.write(0, 0, 'Well')
        styles = dict()
        for well, (color, treatment, concentration) in well_info[nombresListAmps[-1]].items():
            styles[treatment] = workbook.add_format({'bg_color': color}) ## pick treatments and colors from last time point
        styles[''] = workbook.add_format({'bg_color': '#00FF00'})## define not-treated condition and color
        for well, j in col_numbers.items():
            sh.write(0, j + 1, well)
            t,c = well_info[nombresListAmps[-1]][well][1:]
            sh.write(1, j + 1, c + ' ' + t if c != '' else t)
        for i, amp_day_data in enumerate(well_data.values()):
            sh.write(i + 2, 0, nombresListAmps[i])
            for well, amps in amp_day_data.items():
                pbar.update(0.5)      # execution time in this step is limited by disk data transfer speed,
                                      # especially the size of the bl and ac waveform.txt files
                if amps != []:
                    sh.write(i + 2, col_numbers[well] + 1, np.nanmean(amps),
                         styles[well_info[nombresListAmps[i]][well][1]])
                else:
                    sh.write(i + 2, col_numbers[well] + 1, None,
                         styles[well_info[nombresListAmps[i]][well][1]])
    # Write normalized worksheets
        nsh = workbook.add_worksheet('Normalized_Mean_Amplitude')
        nsh.write(1, 0, 'Time')
        nsh.write(0, 0, 'Well')
        for well, j in col_numbers.items():
            nsh.write(0, j + 1, well)
            t,c = well_info[nombresListAmps[-1]][well][1:]
            nsh.write(1, j + 1, c + ' ' + t if c != '' else t)
        for i, amp_day_data in enumerate(well_data.values()):
            nsh.write(i + 2, 0, nombresListAmps[i])
            for well, amps in amp_day_data.items():
                pbar.update(0.5)
                if amps != [] and well_data[baselines[0]][well] != []:
                    nsh.write(i + 2, col_numbers[well] + 1, np.nanmean(amps)/np.nanmean(well_data[baselines[0]][well]),
                         styles[well_info[nombresListAmps[i]][well][1]])
                else:
                    nsh.write(i + 2, col_numbers[well] + 1, None,
                         styles[well_info[nombresListAmps[i]][well][1]])
end = time.time()
print('Created "mean amplitude.xlsx" Workbook,',f"Runtime of the program was {end - start} seconds")
# No RuntimeWarnings means no division of zero or by zero has been performed and excel file will be clean from formula errors

In [ ]:
# Move mean amplitude.xlsx sheets to the two first positions of final tables.xlsx

wb1 = xl.load_workbook(filename='amps.xlsx')
meanamps = wb1.worksheets[0]
nmeanamps = wb1.worksheets[1]

merged = xl.load_workbook(filename=excelName)
copymeanamps = merged.create_sheet(meanamps.title, index=0)
copyNmeanamps = merged.create_sheet(nmeanamps.title, index=1)

with tqdm_notebook(ncols=900, desc="Moving amplitudes cells to final excel " + str(time.ctime(time.time()))) as pbar:
    for row in meanamps:
        for cell in row:
            new_cell = copymeanamps.cell(row=cell.row, column=cell.col_idx,
                    value= cell.value)
            if cell.has_style:
                new_cell.font = copy(cell.font)
                new_cell.border = copy(cell.border)
                new_cell.fill = copy(cell.fill)
                new_cell.number_format = copy(cell.number_format)
                new_cell.protection = copy(cell.protection)
                new_cell.alignment = copy(cell.alignment)
                pbar.update(0.5)
    for row in nmeanamps:
        for cell in row:
            new_cell = copyNmeanamps.cell(row=cell.row, column=cell.col_idx,
                    value= cell.value)
            if cell.has_style:
                new_cell.font = copy(cell.font)
                new_cell.border = copy(cell.border)
                new_cell.fill = copy(cell.fill)
                new_cell.number_format = copy(cell.number_format)
                new_cell.protection = copy(cell.protection)
                new_cell.alignment = copy(cell.alignment)
                pbar.update(0.5)
    ### Rename normalized sheet names
    for i in merged:
        if i.title[0] == 'n':
            i.title = re.sub(r' \([^()]*\)', '', i.title)

merged.active = 0
merged.save(excelName)
os.remove('amps.xlsx')

In [ ]:
# Match treatments with their color using the last time point's color-treatment key-value pair.
# Check the result is correct and use next code cell after manually correcting the excel workbook info if not
co = {}
for color, treatm in zip(day_data['Well Coloring'], day_data['Treatment']):
    co[treatm] = color
temp = {v : k for k, v in co.items()}
res = {v : k for k, v in temp.items()}
print(res)
#res['Control']

In [ ]:
# Read excel workbook to a pandas dataframe
df = pd.read_excel(excelName, index_col=0, header=1, sheet_name=None)

In [ ]:
### Solution for when the treatment names are wrong in the .csv files but correct in the DataFrame after correction
### and the treatment-color pairs must be gotten from the dataframe (using 'Number of Spikes' sheet)

tem = list(df['Number_of_Spikes'].columns)
col_headers = []
# ran = [format(x, '0') for x in range(0,10)]
for e in tem:
    if e.count('.') > 1:
        col_headers.append(e.split(' ', maxsplit=1)[0]+ ' ' + e.split(' ', maxsplit=1)[1].split('.')[0])
    elif '.' not in e:
        col_headers.append(e)
    elif '.' in e[:3] and '.' not in e[-1:-3]:
        col_headers.append(e)
    else:
        col_headers.append(e.split('.')[0])
        
print(col_headers)
wob = xl.load_workbook(excelName, data_only = True)
sh = wob['Number_of_Spikes']
hex_colors = []
for column in range(2, sh.max_column+1):
    hex_colors.append(sh.cell(3,column).fill.start_color.index.replace('FF','#',1))
co = {}
for color, treatm in zip(hex_colors, col_headers):
    co[treatm] = color
temp = {v : k for k, v in co.items()}
res = {v : k for k, v in temp.items()}
print(res)
print ('HEX =', hex_colors) 
#print('RGB =', tuple(int(hex_colors[i:i+2], 16) for i in (0, 2, 4))) # Color in RGB

In [ ]:
# # Create parameter data and SEM averages for each treatment directly from pandas dataframe content
parameters = dict()
sem = dict()

for metric_name, df_metric in tqdm_notebook(df.items(), desc = 'Creating "parameters" and "sem" data dictionaries on ' + str(time.ctime(time.time()))):
    parameters[metric_name] = {k: [] for k in res}
    sem[metric_name] = {k: [] for k in res}
    for day in df_metric.index:
        for trt in parameters[metric_name]:
            vals_over_trt = np.array([df_metric.loc[day][key] for key in df_metric.keys() if key.startswith(trt)])
            with np.errstate(invalid='ignore'):
                parameters[metric_name][trt].append(np.nanmean(vals_over_trt))
                sem[metric_name][trt].append(stats.sem(vals_over_trt, axis=0, ddof=1, nan_policy='omit'))

In [ ]:
# Filter multiple instances of bl and ac if extant, keep only the first one of each and discard those with a number in the third position (ac1, ac2...)

new_realistic_x_axis = {}
for k, v in realistic_x_axis.items():
    if 'bl' in k and not bool(re.search(r'\d', k[2])):
        new_realistic_x_axis[k] = realistic_x_axis[k]
    elif 'ac' in k and not bool(re.search(r'\d', k[2])):
        new_realistic_x_axis[k] = realistic_x_axis[k]
    elif 'bl' in k and bool(re.search(r'\d', k[2])):
        pass
    elif 'ac' in k and bool(re.search(r'\d', k[2])):
        pass
    else:
        new_realistic_x_axis[k] = realistic_x_axis[k]
print('\n'.join(list(new_realistic_x_axis.keys())[:6]), '\nEtc...')

In [ ]:
### Create a composition with all the charts using add_gridspec and add_subplot
# plt.style.use('fivethirtyeight') 

# # Make a deepcopy of df to avoid modifying the original df
# local_df = deepcopy(df)

# # Ignore these warnings:
# warnings.filterwarnings(action='default',
#                         message='(.*divide by zero.*|.*Mean of empty slice.*|.*Degrees of freedom.*|.*invalid value.*|.*Epsilon values.*)')

# # Create parameter data and SEM averages for each treatment directly from pandas dataframe content
# two_way_rm_anovas = {}
# ancovas = {}
# two_way_unb_anova = {}
# # three_way_anova = {}
# tukeys_test = {}
# levenes_test = {}
# parameters = dict()
# sem = dict()
        
# # with interactive plotting (refreshes itself and shows individual plots after every iteration of the loop)
# with plt.ion():
#     for metric_name, df_metric in tqdm_notebook(local_df.items(), desc = 'Plotting metrics on ' + str(time.ctime(time.time()))):
#     ### Extract numbers and calculate Average and SEM for each treatment group and time point before the local_df DataFrame gets altered
# #         metric_name = 'Network_Bursts_Ignored_Flag'
# #         df_metric = local_df['Network_Bursts_Ignored_Flag']
#         print(metric_name)
#         parameters[metric_name] = {k: [] for k in res}
#         sem[metric_name] = {k: [] for k in res}
#         for day in df_metric.index:
#             for trt in parameters[metric_name]:
#                 vals_over_trt = np.array([df_metric.loc[day][key] for key in df_metric.keys() if key.startswith(trt)])
#                 with np.errstate(invalid='ignore'):
#                     parameters[metric_name][trt].append(np.nanmean(vals_over_trt))
#                     sem[metric_name][trt].append(stats.sem(vals_over_trt, axis=0, ddof=1, nan_policy='omit'))
                    
#         # Create grid
#         slide = plt.figure(constrained_layout=False, figsize=(25,35), dpi=100, facecolor = 'white')
#         #         gs1 = slide.add_gridspec(nrows=1, ncols=1, left=0, right=1, wspace=0.1, hspace=0.1) # To place the average + SEM line broken axes plots
#         gs2 = slide.add_gridspec(nrows=7, ncols=3, left=0, right=1, top=1, bottom=0,
#                                  height_ratios=[1.5, 0.5, 0.5, 0.5, 1, 1, 1], width_ratios=[1,1,1], wspace=0.2, hspace=0.3) # For the statistical test and data assessment plots
#         #         slide_ax0 = slide.add_subplot(gs1[:, :])
#         slide_ax1 = slide.add_subplot(gs2[0, :2])
#         slide_ax1_1 = slide.add_subplot(gs2[0, 2])
#         slide_ax2 = slide.add_subplot(gs2[1, :])
#         slide_ax3 = slide.add_subplot(gs2[2, :])
#         slide_ax9 = slide.add_subplot(gs2[3, :])
#         slide_ax4 = slide.add_subplot(gs2[4, :])
#         slide_ax5 = slide.add_subplot(gs2[5, 0])
#         slide_ax5_1 = slide.add_subplot(gs2[5, 1:])
#         slide_ax6 = slide.add_subplot(gs2[6, 0])
#         slide_ax7 = slide.add_subplot(gs2[6, 1])
#         slide_ax8 = slide.add_subplot(gs2[6, 2])

#         allaxes = slide.get_axes()
        
#         # Create a long-format table for statistical calculations
#         cc = pd.DataFrame(local_df[metric_name])
#         cc.columns.name = 'Treatment'
#         cc.index.name = 'Hours'
#         cc = cc.rename({c: c + '.0' for c in local_df[metric_name].columns if c in res}, axis=1, inplace=False)
#         cc.columns = metric_name + '_' + cc.columns
        
#         cc_lineplot = deepcopy(cc)
#         cc_lineplot.index.name = 'Time'
#         cc_lineplot['Time'] = cc_lineplot.index
#         #     cc = cc.drop(cc.columns[[cc.columns.get_loc(c) for c in cc.columns if 'Inactive' in c or 'Dead' in c or 'Untreated' in c]], axis=1, inplace=False)
#         cc_lineplot = pd.wide_to_long(cc_lineplot, stubnames = metric_name, i = 'Time', j = 'Treatment.Well', sep='_',
#                              suffix='({}.\\d{{1,2}})'.format('.\\d{1,2}|'.join(res.keys())).replace('/', '\\'+'/').replace('+','\\'+'+').replace('%', '\\'+'%'))
#         #     cc.to_csv(path_or_buf=r'C:\Users\lab\Desktop\DataFrame.csv', encoding='UTF-8-sig') # export active DataFrame to a .csv file on the Desktop
#         cc_lineplot = cc_lineplot.reset_index(inplace=False)
#         cc_lineplot[['Treatment','Well']] = cc_lineplot['Treatment.Well'].str.split(".", n=1, expand=True)
#         del cc_lineplot['Treatment.Well']
        
#         cc['Hours'] = cc.index
#         cc['Hours'] = list(realistic_x_axis.values())
#     #     cc = cc.drop(cc.columns[[cc.columns.get_loc(c) for c in cc.columns if 'Inactive' in c or 'Dead' in c or 'Untreated' in c]], axis=1, inplace=False)
#         cc = pd.wide_to_long(cc, stubnames = metric_name, i = 'Hours', j = 'Treatment.Well', sep='_',
#                              suffix='({}.\\d{{1,2}})'.format('.\\d{1,2}|'.join(res.keys())).replace('/', '\\'+'/').replace('+','\\'+'+').replace('%', '\\'+'%'))
#     #     cc.to_csv(path_or_buf=r'C:\Users\lab\Desktop\DataFrame.csv', encoding='UTF-8-sig') # export active DataFrame to a .csv file on the Desktop
#         cc = cc.reset_index(inplace=False)
#         cc[['Treatment','Well']] = cc['Treatment.Well'].str.split(".", n=1, expand=True)
#         del cc['Treatment.Well']
#     #     cc.columns = cc.columns.str.replace(" ", "_")
#     #     cc.columns.str.replace("[_(μV)]", "",regex='False')
#     #     cc = cc.replace(np.nan, 0, regex=True) # replace all NaN values with 0 in the long-format DataFrame

# # End of DataFrame formatting

# # Perform statistical tests for each metric/worksheet in the excel

#     #     cc_full = cc.replace(np.nan, 0, regex=True)
#         try:            
#             two_way_rm_anovas[metric_name] = pg.rm_anova(cc, dv=metric_name, within=['Hours', 'Treatment'], subject='Well',
#                                                          detailed=True, correction=True, effsize='n2').round(6)

# #             three_way_anova[metric_name] = pg.anova(cc, dv=metric_name, between=['Hours', 'Treatment', 'Well'],
# #                                                     detailed=True, ss_type=3, effsize="n2").round(6) # We do not usually have the appropiate data for a three-way ANOVA
#         except KeyError as e:
#             two_way_rm_anovas[metric_name] = pd.DataFrame([['NaN', 'NaN'], ['NaN', 'NaN'], ['NaN', 'NaN']], columns = ['no_data0', 'no_data1'])
#             print('Cannot perform two-way repeated measures Anova on {}: {}'.format(metric_name.replace(' ', '_'), e))
        
#         try:
#             two_way_unb_anova[metric_name] = pg.anova(cc, dv=metric_name, between=['Hours', 'Treatment'],
#                                                       detailed=True, effsize="n2").round(6)
#         except (KeyError, AssertionError) as e:
#             two_way_unb_anova[metric_name] = pd.DataFrame([['NaN', 'NaN'], ['NaN', 'NaN'], ['NaN', 'NaN']], columns = ['no_data0', 'no_data1'])
#             print('Cannot perform unbalanced two-way Anova on {}: {}'.format(metric_name.replace(' ', '_'), e))
            
#         try:
#             ancovas[metric_name] = pg.ancova(cc, dv=metric_name, between='Treatment', covar = 'Hours',
#                                              effsize='n2').round(6)
#         except (KeyError, ValueError) as e:
#             ancovas[metric_name] = pd.DataFrame([['NaN', 'NaN'], ['NaN', 'NaN'], ['NaN', 'NaN']], columns = ['no_data0', 'no_data1'])
#             print('Cannot perform ANCOVA on {}: {}'.format(metric_name.replace(' ', '_'), e))
        
#         try:
#             tukeys_test[metric_name] = pg.pairwise_tukey(cc, dv=metric_name, between='Treatment', effsize='eta-square')
#         except (KeyError, ValueError) as e:
#             tukeys_test[metric_name] = pd.DataFrame([['NaN', 'NaN'], ['NaN', 'NaN'], ['NaN', 'NaN']], columns = ['no_data0', 'no_data1'])
#             print('Cannot perform Tukey\'s test on {}: {}'.format(metric_name.replace(' ', '_'), e))

#         strip_plot = sns.stripplot(x='Treatment', y=metric_name, hue='Well', 
#                                    data=cc, ax=slide_ax1, size=9, jitter=True, edgecolor='black', linewidth=0.5, dodge=True)
#         box_plot = sns.boxplot(x='Treatment', y=metric_name, hue='Treatment', data=cc,
#                                ax=slide_ax1, dodge=False, palette = [v for k, v in sorted(res.items())])
#     #     box_plot.legend(bbox_to_anchor=(1.03, 1), borderaxespad=0., labelspacing = 1, fontsize=17)
#         bar_plot = sns.barplot(x='Treatment', y=metric_name, hue='Treatment', units='Well', data=cc, ax=slide_ax1_1,
#                                dodge=False, ci=95, errcolor='black', errwidth=1.5, capsize=0.25, palette = [v for k, v in sorted(res.items())])
#         plt.setp(slide_ax1_1.xaxis.get_majorticklabels(), rotation=-10, ha='left', rotation_mode='anchor')
#         slide_ax1.set_xlabel('Treatment', loc='center', labelpad=20, fontsize=18)
#         slide_ax1.set_title(metric_name, pad=20, fontsize=20)
#         slide_ax1.tick_params(axis='both', which='major', labelsize=17)
#         slide_ax1.set_ylabel(metric_name, labelpad=25, fontsize=20)
#         slide_ax1_1.set_xlabel('Treatment', loc='center', labelpad=20, fontsize=18)
#         slide_ax1_1.set_title(metric_name, pad=20, fontsize=20)
#         slide_ax1_1.tick_params(axis='both', which='major', labelsize=17)
#         slide_ax1_1.set_ylabel(metric_name, labelpad=25, fontsize=20)

#         # Add repeated measures two-way ANOVA results table to the graph
#         two_way_rm_anovas_text = []
#         for row in range(len(two_way_rm_anovas[metric_name])):
#             two_way_rm_anovas_text.append(two_way_rm_anovas[metric_name].iloc[row])
#         table1 = slide_ax2.table(cellText=two_way_rm_anovas_text, colLabels=two_way_rm_anovas[metric_name].columns, loc='bottom', bbox=[0, -0.1, 1, 1])
#         slide_ax2.set_title('Two-way repeated measures Anova test within time and treatment groups for {}'.format(metric_name),
#                            fontsize=18, pad=0)
#         slide_ax2.axis('off')
# #         table1.auto_set_font_size(False)
# #         table1.set_fontsize(12)

#         # Add unbalanced two-way ANOVA table to the graph
#         two_way_unb_anova_text = []
#         for row in range(len(two_way_unb_anova[metric_name])):
#             two_way_unb_anova_text.append(two_way_unb_anova[metric_name].iloc[row])
#         table2 = slide_ax3.table(cellText=two_way_unb_anova_text, colLabels=two_way_unb_anova[metric_name].columns, loc='bottom', bbox=[0, 0, 1, 1])
#         slide_ax3.set_title('Two-way unbalanced samples Anova test within time and treatment groups for {}'.format(metric_name),
#                            fontsize=18, pad=20)
#         slide_ax3.axis('off')
# #         table2.auto_set_font_size(False)
# #         table2.set_fontsize(11)

#         # Add unbalanced two-way ANOVA table to the graph
#         ancova_text = []
#         for row in range(len(ancovas[metric_name])):
#             ancova_text.append(ancovas[metric_name].iloc[row])
#         table5 = slide_ax9.table(cellText=ancova_text, colLabels=ancovas[metric_name].columns, loc='bottom', bbox=[0, 0, 1, 1])
#         slide_ax9.set_title('Analisis of covariance between treatments ~ Time + Replicates for {}'.format(metric_name),
#                            fontsize=18, pad=20)
#         slide_ax9.axis('off')
# #         table5.auto_set_font_size(False)
# #         table5.set_fontsize(11)

#            # Add Tukey's test table to the graph
#         tukeys_text = []
#         for row in range(len(tukeys_test[metric_name])):
#             tukeys_text.append(tukeys_test[metric_name].iloc[row])
#         table3 = slide_ax4.table(cellText=tukeys_text, colLabels=tukeys_test[metric_name].columns, loc='bottom', bbox=[0, 0, 1, 1])
# #         table3.auto_set_font_size(False)
#         slide_ax4.set_title('Pairwise Tukey\'s post-hoc test (table) within treatment groups for {}'.format(metric_name),
#                            fontsize=18, pad=20)
#         slide_ax4.axis('off')
# #         table3.set_fontsize(11)

#         interaction_plot(x=cc['Well'], trace=cc['Treatment'], response=cc[metric_name], ax=slide_ax5, plottype='b', colors=[v for k, v in sorted(res.items())])
#         slide_ax5.set_title('{} on each replicate (well)'.format(metric_name),
#                            fontsize=18, pad=20)
        
# #         interaction_plot(x=cc['Hours'], trace=cc['Treatment'], response=cc[metric_name], ax=slide_ax5_1, plottype='b', colors=[v for k, v in sorted(res.items())])
#         sns.lineplot(x='Time', y=metric_name, data=cc_lineplot, hue='Treatment', palette=[v for k, v in sorted(res.items())],
#                              estimator='mean', ax=slide_ax5_1, err_style='bars', n_boot=1000, sort=True, ci=95)
#         #         fancy_lineplot(name, mean_metr, sem, res, new_realistic_x_axis, df)
#         slide_ax5_1.set_title('Treatment effect on {} over time (95% CI)'.format(metric_name),
#                              fontsize=18, pad=20)
#         plt.setp(slide_ax5_1.xaxis.get_majorticklabels(), rotation=-25, ha='left', rotation_mode='anchor')
# #         field = "Hours"
# #         time_order = [list(df[metric_name].index)]
# #         cc.set_index(field).loc[time_order].plot

#         try:
#             residuals = stat()
#             residuals.anova_stat(df=cc, res_var='Q(\'{}\')'.format(metric_name),
#                              anova_model='{}~C(Treatment)+C(Hours)+C(Treatment):C(Hours)'.format('Q(\'{}\')'.format(metric_name)))
#             pg.qqplot(residuals.anova_std_residuals, dist='norm', confidence=False, ax = slide_ax6)
#             pg.qqplot(residuals.anova_std_residuals, dist='expon', ax=slide_ax7)
#             # histogram
#             slide_ax8 = plt.hist(residuals.anova_model_out.resid, bins='auto', histtype='bar', ec='k', zorder=1) 
#             plt.xlabel('Residuals')
#             plt.ylabel('Frequency')
#             plt.title('Histogram of residuals\' distribution')

#             # Shapiro-Wilkins test for normal distribution of residuals 
#             # (Null hypothesis: data were drawn from a normal distribution)
#             w, pvalue = stats.shapiro(residuals.anova_model_out.resid)
#             textstr1 = '\n'.join((r'$\bf{Shapiro-Wilkins test}$',
#                                   r'$\mathrm{w}=%.30f$' % (w, ),
#                                   r'$\mathrm{p-value}=%.30f$' % (pvalue, )))
#             # these are matplotlib.patch.Patch properties for the text box
#             hist_props = dict(boxstyle='round', facecolor='wheat', alpha=.5)
#             # add text box with the data to the plot
#             plt.text(0.555, 0.99, textstr1, transform=allaxes[10].transAxes, fontsize=9,
#                     verticalalignment='top', bbox=hist_props)

#             # Levene's test for homogeneity of variances
#             residuals.levene(df=cc, res_var=metric_name, xfac_var=['Treatment', 'Hours'])   
#             lev_text = []
#             levene_cell_colors = [["wheat","wheat"],["wheat","wheat"],[ "wheat","wheat"]]
#             for row in range(len(residuals.levene_summary)):
#                 lev_text.append(residuals.levene_summary.iloc[row])
#             table4 = plt.table(cellText=lev_text, colLabels=residuals.levene_summary.columns,
#                               transform=allaxes[10].transAxes, loc='left', edges='closed',
#                               cellColours=levene_cell_colors, colColours=["gainsboro","gainsboro"],
#                                bbox=[0,0.75,0.25,0.25], zorder=2, alpha=.5)
#             # Make table4 semitransparent
#             for cell in table4._cells:
#                 table4._cells[cell].set_alpha(.5)
#             plt.text(0.01, 1.035, 'Levene\'s test summary', transform=allaxes[10].transAxes,
#                     verticalalignment='top', fontsize=9, weight='bold')
#         except ValueError as e:
#             print('Cannot calculate residuals of {} distribution: {} array is zero-size'.format(metric_name.replace(' ', '_'), e))

#         # Save the figure within the loop
#         if os.path.isdir('Descriptive statistics and plots') == False:
#             os.mkdir('Descriptive statistics and plots')
#         slide.savefig('Descriptive statistics and plots\\' + metric_name, edgecolor='none', transparent=False)
        
#         plt.show(close=None, block=None)
# #         plt.close(slide)

plt.close("all") # Close all figures and do not display any inline after running the loop

In [ ]:
# Some plotting presets

titles_and_axes_on = FontProperties(family='Arial', style='normal', weight=250, stretch='normal', size=48)
legends_on = FontProperties(family='Arial', style='normal', weight=250, stretch='normal', size=48)
devel = []
overnight = []
after = []
post_on = []
for k, val in new_realistic_x_axis.items():
    if 'bl' in k:
        break
    devel.append(k)

for k, val in new_realistic_x_axis.items():
    if 'bl' in k or 'ac' in k or 'on' in k or 'Pretreatment' in k:
        overnight.append(k)
    else:
        break

overnight.append(df[name].index.values[len(overnight)])
after = df[name].index.values[len(devel):]
post_on = df[name].index.values[len(devel)+len(overnight):]
print('Stablished plot presets on ' + str(time.ctime(time.time())))
    
# Plots post-treatment values with SEM whiskers in a broken (split) x-axis

with plt.ion():
    for name, mean_metr in tqdm_notebook(parameters.items(), total=len(parameters.keys()),
                                         desc='Plotting and saving graphs on ' + str(time.ctime(time.time()))):
        if post_on.size == 0:
            fig1, axs1 = plt.subplots(1, 1, figsize=(35, 15), gridspec_kw=dict(hspace=0, wspace=0), sharex='none', sharey=True)
            axs1.patch.set_alpha(0.0)
            realistic_x = []
            for trt, row in mean_metr.items():
                x = [df[name].index.get_loc(ind) for ind in after]
                realistic_x = [list(new_realistic_x_axis.values())[i] for i in x]
                axs1.errorbar(realistic_x, np.array(row)[x], yerr = np.array(sem[name][trt])[x], label=trt,
                              elinewidth=2, linewidth=2.25, color='black', marker='o', markersize='24', zorder=2,
                              barsabove=True, markeredgewidth='2.5', markeredgecolor='black',
                              markerfacecolor=res[trt],capsize=10, capthick=2, ecolor=res[trt])
                handles, labels = axs1.get_legend_handles_labels()
                handles = [h[0] if isinstance(h, container.ErrorbarContainer) else h for h in handles]
                axs1.legend(handles, labels, bbox_to_anchor=(0.75, 1.1), loc="lower left", framealpha=0, prop=legends_on)
        else:
            fig1, (axs1,axs2) = plt.subplots(1, 2, figsize=(35, 15),
                                             gridspec_kw=dict({'width_ratios': [7, 3]},hspace=0, wspace=0.04),
                                             sharex='none', sharey=True, facecolor='w')
            axs1.patch.set_alpha(0.0)
            axs2.patch.set_alpha(0.0)
            realistic_x = []
            for trt, row in mean_metr.items():
                x = [df[name].index.get_loc(ind) for ind in after]
                realistic_x = [list(new_realistic_x_axis.values())[i] for i in x]
                axs1.errorbar(realistic_x, np.array(row)[x], yerr = np.array(sem[name][trt])[x], label=trt,
                              elinewidth=2, linewidth=2.25, color='black', marker='o', markersize='24', zorder=2,
                              barsabove=True, markeredgewidth='2.5', markeredgecolor='black', markerfacecolor=res[trt],
                              capsize=10, capthick=2, ecolor=res[trt])
                axs2.errorbar(realistic_x, np.array(row)[x], yerr = np.array(sem[name][trt])[x], label=trt,
                              elinewidth=2, linewidth=2.25, color='black', marker='o', markersize='24', zorder=2,
                              barsabove=True, markeredgewidth='2.5', markeredgecolor='black', markerfacecolor=res[trt],
                              capsize=10, capthick=2, ecolor=res[trt])
            axs2.set_xlim(new_realistic_x_axis[post_on[0]] - 5, new_realistic_x_axis[post_on[-1]] + 8)
            plt.setp(axs2, xticks=np.arange(round(new_realistic_x_axis[post_on[0]]), round(new_realistic_x_axis[post_on[-1]] + 50), 50))
            axs2.tick_params(axis='x', length=25, width=3.5, pad=12)
            plt.setp(axs2.xaxis.get_majorticklabels(), rotation=0, fontproperties=titles_and_axes_on)
            axs2.spines['left'].set_visible(False)
            axs2.spines['bottom'].set_linewidth(3.5)
            axs2.spines['top'].set_visible(False)
            axs2.spines['right'].set_visible(False)
            axs2.yaxis.set_visible(False)
            axs2.grid(False)
            d = .025 # how big to make the diagonal lines in axes coordinates
            kwargs = dict(transform=axs1.transAxes, color='k', clip_on=False, linewidth=3.5)
            axs1.plot((1,1), (-d,+d), **kwargs)
            kwargs.update(transform=axs2.transAxes)
            axs2.plot((0,0), (-d,+d), **kwargs)
            handles, labels = axs1.get_legend_handles_labels()
            handles = [h[0] if isinstance(h, container.ErrorbarContainer) else h for h in handles]
            axs1.legend(handles, labels, ncol=2, labelspacing=1.5, columnspacing=8, bbox_to_anchor=(0.75, 1.05), loc="lower center", framealpha=0, prop=legends_on)

        # Settings common to both cases
        axs1.set_xlim(new_realistic_x_axis[overnight[0]]-0.5, new_realistic_x_axis[overnight[-1]] + 0.25)
        plt.setp(axs1, xticks=np.arange(0, round(new_realistic_x_axis[overnight[-1]]), 1))
        plt.setp(axs1.yaxis.get_majorticklabels(), fontproperties=titles_and_axes_on)
        plt.setp(axs1.xaxis.get_majorticklabels(), rotation=0, fontproperties=titles_and_axes_on)
        axs1.tick_params(axis='both', labelright='off', length=25, width=3.5, pad=12)
        axs1.spines['left'].set_linewidth(3.5)
        axs1.spines['bottom'].set_linewidth(3.5)
        axs1.spines['top'].set_visible(False)
        axs1.spines['right'].set_visible(False)
        axs1.yaxis.tick_left()
        axs1.grid(False)
        if name[0] == 'n':
            name = re.sub('^n', 'Normalized ', name) ### use '^n' to replace the first lowercase n by 'Normalized',there is no length limit unlike for excel sheet names
            name = re.sub(r' \([^()]*\)', '', name) ### use ' (*' as a marker to delete units (all units in names are inside parentheses)
        fig1.text(0.5, -0.02, 'Time (hours)', ha='center', fontproperties=titles_and_axes_on)
        axs1.set_ylabel(name, labelpad=35, fontproperties=titles_and_axes_on)
        # Set bottom ylimit to 0 if ylim[0] > 0
        ymin, ymax = axs1.get_ylim()
        if ymin > 0:
            axs1.set_ylim(ymin=0, ymax=ymax*1.1)
        else:
            pass
        axs1.annotate('', xy=(new_realistic_x_axis[[k for k in new_realistic_x_axis.keys() if 'ac' in k][0]], axs1.get_ylim()[0]),
                      xytext=(new_realistic_x_axis[[k for k in new_realistic_x_axis.keys() if 'ac' in k][0]], axs1.get_ylim()[1]), rotation=0,
                      ha='center', va='center', annotation_clip=False, zorder=1,
                      arrowprops={"color" : "purple", "arrowstyle" : "-", "linestyle" : ":", "linewidth" : 2, "shrinkA": 0, "shrinkB": 0})

        axs1.text(new_realistic_x_axis[[k for k in new_realistic_x_axis.keys() if 'ac' in k][0]]+0.15, axs1.get_ylim()[1]*0.89,
                  'DIV'+str(day_of_treatment), rotation=90, fontname='Arial', color='black', fontsize=40,
                  ha='left', va='baseline') # if text separates more than 0.2 from the line is because of autoshrink
        if post_on.size > 0:
            axs2.annotate('', xy=(new_realistic_x_axis[post_on[0]], axs2.get_ylim()[0]),
                          xytext=(new_realistic_x_axis[post_on[0]], axs2.get_ylim()[1]), rotation=0,
                          fontsize=37, fontname='Arial', color='black', ha='center', va='center', annotation_clip=False, zorder=1,
                          arrowprops={"color" : "purple", "arrowstyle" : "-", "linestyle" : ":", "linewidth" : 2, "shrinkA": 0, "shrinkB": 0})

            axs2.text(new_realistic_x_axis[post_on[0]]+5, axs1.get_ylim()[1]*0.89,
                      post_on[0].upper(), rotation=90, fontname='Arial', color='black', fontsize=40, ha='left', va='baseline')
            axs2.annotate('', xy=(new_realistic_x_axis[post_on[-1]], axs2.get_ylim()[0]),
                          xytext=(new_realistic_x_axis[post_on[-1]], axs2.get_ylim()[1]), rotation=0,
                          fontsize=37, fontname='Arial', color='black', ha='center', va='center', annotation_clip=False, zorder=1,
                          arrowprops={"color" : "purple", "arrowstyle" : "-", "linestyle" : ":", "linewidth" : 2, "shrinkA": 0, "shrinkB": 0})

            axs2.text(new_realistic_x_axis[post_on[-1]]+5, axs1.get_ylim()[1]*0.89,
                      post_on[-1].upper(), rotation=90, fontname='Arial', color='black', fontsize=40, ha='left', va='baseline')

            if os.path.isdir('Figures') == False:
                os.mkdir('Figures')
            fig1.savefig('Figures\\' + name, edgecolor='none', transparent=False)

            plt.show(close=None, block=None)

    plt.close("all") # Close all figures and do not display any inline after running the loop

    # Move all .csv files to "Sorted analysis files" folder

if os.path.isdir('Sorted analysis files') == False:
    os.mkdir('Sorted analysis files')
for fl in os.listdir():
    if fl.endswith('.csv') or fl.endswith('.txt'):
        shutil.move(os.path.join(path, fl), os.path.join(path, 'Sorted analysis files', fl))